### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

<<참고 : https://colah.github.io/posts/2015-08-Understanding-LSTMs/ >>
https://link.springer.com/article/10.1007/s11063-009-9096-2

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [2]:
# Sequence data
x = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(x, (-1,4,1))  # (6,4,1)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

### [1] SimpleRNN
#### 가장 간단한 형태의 RNN 

In [3]:
#Q1)  SimpleRNN모델 구현하자.  

model = Sequential([
# X  : (6, 4, 1)   reshape한 데이터
# Wx : (1, 300)   ,  (입력차원 1, 뉴런 300개) 입력 가중치 = 파라미터 개수 = 1*300=300개
# Wh : (300, 300) ,  (유닛개수, 숨겨진 상태)  숨겨진 상태의 가중치.  그래서 파라미터 수는 300*300=90,000
# b  : (300) ,
# 총 param : 90600      = Wx 300 + Wh 90,000 + b 300 = 90,600
# OUt (None, 4, 300)  
    SimpleRNN(units=300, return_sequences=True, input_shape=[4,1]),
    
# X  : (None, 4, 300)   
# Wx : (300, 300),  앞 레이어에서 유닛 300개로 받았기 때문에, 입력차원이 300이다.
# Wh : (300, 300),      위와 같음
# b  : (300),
# 총 param : 180300   , OUt (None, 300)  
    SimpleRNN(units=300),
    Dense(1)      # 출력 차원이 1, 가중치 형태 = (300, 1) = 300*1 + 1 = 301
])

model.compile(optimizer='adam',loss='mse')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 300)            90600     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 271201 (1.03 MB)
Trainable params: 271201 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
model.fit(x_data, y_data, epochs=100, verbose=1)
print(model.predict(x_data))

Epoch 1/100

1/1 [==============================] - 3s 3s/step - loss: 53.7663
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 1.8309
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 5.8359
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 8.6183
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 4.8950
Epoch 6/100
1/1 [==============================] - 0s 13ms/step - loss: 1.6185
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 2.0275
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 3.6434
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 3.4279
Epoch 10/100
1/1 [==============================] - 0s 11ms/step - loss: 2.0649
Epoch 11/100
1/1 [==============================] - 0s 13ms/step - loss: 1.0392
Epoch 12/100
1/1 [==============================] - 0s 12ms/step - loss: 0.8203
Epoch 13/100
1/1 [==============================]

1/1 [==============================] - 1s 504ms/step
[[4.01417  ]
 [4.96672  ]
 [5.9981103]
 [7.088033 ]
 [8.073884 ]
 [8.867755 ]]


In [5]:
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 82ms/step
[[9.459349]]
1/1 [==============================] - 0s 81ms/step
[[0.7824381]]


In [6]:
model.evaluate(x_data, y_data)

1/1 [==============================] - 1s 690ms/step - loss: 0.0053


0.005334893707185984

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [7]:
#Q2) RNN 순환 신경망 구현  : LSTM
model = Sequential([   
 #X :  (6,4,1) 
# Wx : (1,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*90600      , OUt (None, 4, 300)
    LSTM(units=300, return_sequences=True, input_shape=[4,1]),  
    
  #X :  None, 4, 300
# Wx : (300,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*180300     , OUt   (None, 300) 
    LSTM(units=300),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 300)            362400    
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dense_1 (Dense)             (None, 1)                 301       
                                                                 
Total params: 1083901 (4.13 MB)
Trainable params: 1083901 (4.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 44.5882
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 38.7813
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 32.9089
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 26.3858
Epoch 5/100
1/1 [==============================] - 0s 25ms/step - loss: 19.0041
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 11.1793
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 4.4054
Epoch 8/100
1/1 [==============================] - 0s 24ms/step - loss: 1.3605
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 3.8046
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 6.7651
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - loss: 6.9216
Epoch 12/100
1/1 [==============================] - 0s 25ms/step - loss: 5.1851
Epoch 13/100
1/1 [===========================

In [9]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 24ms/step
[[8.927953]]
1/1 [==============================] - 0s 36ms/step
[[1.2589666]]


In [10]:
model.evaluate(x_data,y_data)

1/1 [==============================] - 1s 996ms/step - loss: 0.1427


0.1427234411239624

### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [11]:
# RNN 순환 신경망 구현  : GRU
model = Sequential([
    GRU(units=300, return_sequences=True, input_shape=[4,1]),
    GRU(units=300),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 300)            272700    
                                                                 
 gru_1 (GRU)                 (None, 300)               541800    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 814801 (3.11 MB)
Trainable params: 814801 (3.11 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 46.8683
Epoch 2/100
1/1 [==============================] - 0s 19ms/step - loss: 30.1131
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 16.6262
Epoch 4/100
1/1 [==============================] - 0s 21ms/step - loss: 6.3786
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 0.9285
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 2.3534
Epoch 7/100
1/1 [==============================] - 0s 19ms/step - loss: 6.2152
Epoch 8/100
1/1 [==============================] - 0s 22ms/step - loss: 6.5697
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 4.4283
Epoch 10/100
1/1 [==============================] - 0s 21ms/step - loss: 2.0281
Epoch 11/100
1/1 [==============================] - 0s 21ms/step - loss: 0.5887
Epoch 12/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2671
Epoch 13/100
1/1 [==============================

In [13]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 22ms/step
[[9.294562]]
1/1 [==============================] - 0s 23ms/step
[[2.1034653]]
